# Tutorial: Classificação Não Supervisionada de Imagens de Sensoriamento Remoto usando CNN

Este notebook apresenta um guia passo a passo sobre como usar Redes Neurais Convolucionais (CNN) para classificação não supervisionada de imagens de sensoriamento remoto do Sentinel-2.

## Conteúdo:
1. Introdução ao Sensoriamento Remoto e CNNs
2. Preparação dos Dados
3. Implementação do Modelo
4. Treinamento e Validação
5. Visualização e Análise dos Resultados

## 1. Introdução

### 1.1 Sensoriamento Remoto
O sensoriamento remoto é a técnica de obter informações sobre objetos ou áreas através de dados coletados por instrumentos que não estão em contato direto com os objetos de investigação. No nosso caso, usaremos imagens do satélite Sentinel-2, que fornece imagens multiespectrais de alta resolução.

### 1.2 Por que usar CNN?
CNNs são especialmente eficazes para processamento de imagens porque:
- Podem aprender hierarquias de características automaticamente
- São invariantes a translações
- Podem capturar padrões espaciais complexos

### 1.3 Abordagem Não Supervisionada
Usaremos um autoencoder com clustering para identificar padrões nas imagens sem necessidade de rótulos, o que é especialmente útil quando não temos dados rotulados disponíveis.

## 2. Preparação do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e configurar nosso ambiente:

In [ ]:
import sys
sys.path.append('..')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from src.data.tiff_loader import TiffLoader
from src.models.unsupervised_cnn import UnsupervisedCNN
import mlflow
import seaborn as sns

# Configurar o estilo dos gráficos
plt.style.use('seaborn')
%matplotlib inline

## 3. Carregamento e Preparação dos Dados

### 3.1 Entendendo as Bandas do Sentinel-2

Neste projeto, usamos três bandas específicas do Sentinel-2:
- **B8 (NIR)**: Infravermelho próximo (842nm) - Útil para análise de vegetação
- **B4 (Red)**: Vermelho (665nm) - Absorção de clorofila
- **B11 (SWIR)**: Infravermelho de onda curta (1610nm) - Sensível à umidade

### 3.2 Carregando os Dados

In [ ]:
# Inicializar o loader
loader = TiffLoader(
    train_val_dir='../data/train',  # Ajuste o caminho conforme necessário
    prediction_dir='../data/predict',
    patch_size=64
)

# Carregar dados de treino e validação
train_ds, val_ds = loader.load_train_val_data(val_split=0.2)

# Preparar os datasets
train_ds = loader.prepare_dataset(train_ds, batch_size=32, shuffle=True, augment=True)
val_ds = loader.prepare_dataset(val_ds, batch_size=32, shuffle=False)

### 3.3 Visualizando os Dados

Vamos visualizar alguns exemplos dos nossos patches:

In [ ]:
def plot_patches(dataset, num_patches=5):
    plt.figure(figsize=(15, 3))
    for i, batch in enumerate(dataset.take(1)):
        for j in range(num_patches):
            plt.subplot(1, num_patches, j+1)
            # Composição RGB usando as bandas disponíveis
            rgb = batch[j].numpy()
            plt.imshow(rgb)
            plt.axis('off')
            plt.title(f'Patch {j+1}')
    plt.tight_layout()

plot_patches(train_ds)

## 4. Implementação e Treinamento do Modelo

### 4.1 Arquitetura do Modelo

Nosso modelo consiste em duas partes principais:
1. **Autoencoder**: Para aprender uma representação compacta dos dados
2. **K-means**: Para clustering no espaço latente

### 4.2 Treinamento do Modelo

In [ ]:
# Inicializar o modelo
model = UnsupervisedCNN(
    input_shape=(64, 64, 3),
    n_clusters=5,
    latent_dim=128,
    experiment_name='tutorial_sentinel2'
)

# Converter dataset para numpy array para treinamento
X_train = np.concatenate([batch.numpy() for batch in train_ds], axis=0)

# Treinar o modelo
model.train(
    X=X_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    model_params={'description': 'Tutorial Sentinel-2'}
)

## 5. Análise dos Resultados

### 5.1 Visualização dos Clusters

In [ ]:
# Fazer predições
predictions = model.predict(X_train)

# Visualizar exemplos de cada cluster
plt.figure(figsize=(15, 3*model.n_clusters))
for cluster in range(model.n_clusters):
    cluster_samples = X_train[predictions == cluster][:5]
    for j, sample in enumerate(cluster_samples):
        plt.subplot(model.n_clusters, 5, cluster*5 + j + 1)
        plt.imshow(sample)
        plt.axis('off')
        if j == 0:
            plt.ylabel(f'Cluster {cluster}')
plt.tight_layout()

### 5.2 Análise do Espaço Latente

Vamos visualizar como as amostras estão distribuídas no espaço latente usando PCA:

In [ ]:
from sklearn.decomposition import PCA

# Obter representações latentes
latent_features = model.encoder.predict(X_train)

# Reduzir dimensionalidade para visualização
pca = PCA(n_components=2)
latent_2d = pca.fit_transform(latent_features)

# Plotar
plt.figure(figsize=(10, 8))
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=predictions, cmap='viridis')
plt.colorbar(scatter)
plt.title('Distribuição das Amostras no Espaço Latente')
plt.xlabel('Primeira Componente Principal')
plt.ylabel('Segunda Componente Principal')

## 6. Interpretação dos Resultados

### 6.1 Características dos Clusters

Cada cluster identificado pode representar diferentes tipos de cobertura do solo ou características da paisagem. Por exemplo:
- Áreas urbanas
- Vegetação densa
- Solo exposto
- Corpos d'água
- Áreas agrícolas

### 6.2 Limitações e Considerações

É importante notar algumas limitações desta abordagem:
1. O número de clusters é definido manualmente
2. A interpretação dos clusters requer conhecimento do domínio
3. A qualidade do clustering pode variar dependendo da representatividade dos dados

## 7. Próximos Passos

Para melhorar os resultados, você pode:
1. Experimentar com diferentes números de clusters
2. Ajustar a arquitetura do autoencoder
3. Incluir mais bandas espectrais
4. Usar técnicas de validação mais robustas
5. Incorporar conhecimento do domínio na interpretação dos resultados